In [0]:
!git clone https://github.com/JuanIsernGhosn/coral-classification

fatal: destination path 'coral-classification' already exists and is not an empty directory.


In [0]:
experiment = "coral-classification-deep-learning/experiments/vgg16_Keras_2/"

In [0]:
# Imports
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.framework import ops
from tensorflow.python.framework import dtypes
import numpy
import math
import pandas as pd
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, Flatten, Activation, merge
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, Adam
import keras.backend as K
from natsort import natsorted, ns
import keras
import PIL
from keras.regularizers import l2
from glob import glob
from keras.applications import vgg16
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [0]:
# Fijar las semillas
seed = 2032 # Semilla del numpy
tf.set_random_seed(seed)# Fijar semilla del keras/tensorflow

epochs = 1000
batch_size = 32

model_name = 'model.h5'

In [0]:
# Train and test directories
train_dir = "coral-classification/coral_img/train/"
test_dir = "coral-classification/coral_img/Test_Mixed/"

In [0]:
# Classes
clases = sorted(os.listdir(train_dir))
print(clases)

x_train = np.array([cv2.imread(os.path.join(train_dir, cl, name)) for cl in clases
           for name in os.listdir(os.path.join(train_dir, cl))])
y_train = np.array([n for n, cl in enumerate(clases)
           for name in os.listdir(os.path.join(train_dir, cl))])

idx = np.random.permutation(len(x_train))
x_train, y_train = x_train[idx], y_train[idx]

test_files = natsorted(os.listdir(test_dir))

x_test = np.array([cv2.imread(os.path.join(test_dir, name)) 
                   for name in test_files])

['ACER', 'APAL', 'CNAT', 'DANT', 'DSTR', 'GORG', 'MALC', 'MCAV', 'MMEA', 'MONT', 'PALY', 'SPO', 'SSID', 'TUNI']


In [0]:

x_train = x_train.astype('float32')
x_train /= 255

x_test = x_test.astype('float32')
x_test /= 255

y_train = to_categorical(y_train, dtype=int)

print(x_train.shape)
print(len(clases))
print(y_train.shape)

(620, 256, 256, 3)
14
(620, 14)


In [0]:
datagen = ImageDataGenerator(
    rotation_range=20,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2,
    fill_mode='nearest'
)

In [0]:
# create the base pre-trained model
base_model = vgg16.VGG16(weights='imagenet',include_top=False, input_shape=(256,256,3))

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
__________

In [0]:
x=base_model.output
x=Conv2D(512, (8, 8), activation='relu')(x)
#x=MaxPooling2D(pool_size=(2, 2))(x)
x=Dropout(rate=0.25)(x)
x=Flatten()(x)
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(14,activation='softmax')(x) #final layer with softmax activation

model=Model(inputs=base_model.input, outputs=preds)

In [0]:
for i,layer in enumerate(model.layers):
  print(i,layer.name)

0 input_1
1 block1_conv1
2 block1_conv2
3 block1_pool
4 block2_conv1
5 block2_conv2
6 block2_pool
7 block3_conv1
8 block3_conv2
9 block3_conv3
10 block3_pool
11 block4_conv1
12 block4_conv2
13 block4_conv3
14 block4_pool
15 block5_conv1
16 block5_conv2
17 block5_conv3
18 block5_pool
19 conv2d_11
20 dropout_3
21 flatten_3
22 dense_5
23 dense_6


In [0]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in model.layers[:19]:
    layer.trainable=False
for layer in model.layers[19:]:
    layer.trainable=True

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer=Adam(lr=0.001), 
              loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
es = EarlyStopping(monitor='val_loss', mode='min', min_delta=0, patience=15)

In [0]:
# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
datagen.fit(x_train)


# fits the model on batches with real-time data augmentation:
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size, subset="training"),
                   steps_per_epoch=int(len(x_train)*0.8) / batch_size, epochs=epochs,
                   validation_data = datagen.flow(x_train, y_train, batch_size=batch_size, subset='validation'),
                   validation_steps = int(len(x_train)*0.2) / batch_size,
                   verbose=1, callbacks=[es])

model.save(model_name)

Epoch 1/1000
16/15 [==============================] - 12s 749ms/step - loss: 0.3025 - acc: 0.9045 - val_loss: 0.4512 - val_acc: 0.8306
Epoch 2/1000
16/15 [==============================] - 10s 639ms/step - loss: 0.2675 - acc: 0.9080 - val_loss: 0.5816 - val_acc: 0.7823
Epoch 3/1000
16/15 [==============================] - 10s 649ms/step - loss: 0.2968 - acc: 0.9006 - val_loss: 0.7186 - val_acc: 0.8065
Epoch 4/1000
16/15 [==============================] - 10s 643ms/step - loss: 0.3054 - acc: 0.8789 - val_loss: 0.5612 - val_acc: 0.8226
Epoch 5/1000
16/15 [==============================] - 10s 637ms/step - loss: 0.3333 - acc: 0.8905 - val_loss: 0.6303 - val_acc: 0.8306
Epoch 6/1000
16/15 [==============================] - 10s 638ms/step - loss: 0.3335 - acc: 0.8770 - val_loss: 0.5942 - val_acc: 0.8145
Epoch 7/1000
16/15 [==============================] - 10s 635ms/step - loss: 0.3234 - acc: 0.9022 - val_loss: 0.6148 - val_acc: 0.8065
Epoch 8/1000
16/15 [==============================] - 1

In [0]:
predictions_test = model.predict(x_test, verbose=1)

data = pd.DataFrame()
data['Id'] = test_files
data['Category'] = predictions_test.argmax(axis=-1)


data.to_csv("envio.csv", index=False)

146/146 [==============================] - 2s 13ms/step


In [0]:
data

,Id,Category
0,aT1.jpg,0
1,aT2.jpg,0
2,aT5.jpg,0
3,aT17.jpg,0
4,aT21.jpg,1
5,aT31.jpg,3
6,aT33.jpg,10
7,aT34.jpg,0
8,aT47.jpg,0
9,aT55.jpg,0
